<a href="https://colab.research.google.com/github/githubpradeep/notebooks/blob/main/RAGatouille.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ragatouille sqlite-utils -q

In [ ]:
!pip uninstall --y faiss-cpu & pip install faiss-gpu -q

In [ ]:
from ragatouille import RAGPretrainedModel
import re
import sqlite_utils

In [ ]:
_tags_re = re.compile(r'<[^>]+>')

def strip_html_tags(text):
    return _tags_re.sub('', text)

In [ ]:
!wget https://datasette.simonwillison.net/simonwillisonblog.db

--2024-01-29 12:27:57--  https://datasette.simonwillison.net/simonwillisonblog.db
Resolving datasette.simonwillison.net (datasette.simonwillison.net)... 173.194.192.121, 2607:f8b0:4001:c62::79
Connecting to datasette.simonwillison.net (datasette.simonwillison.net)|173.194.192.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘simonwillisonblog.db’

simonwillisonblog.d     [        <=>         ]  83.35M  16.4MB/s    in 5.2s    

2024-01-29 12:28:02 (16.1 MB/s) - ‘simonwillisonblog.db’ saved [87400448]



In [ ]:
def go():
    db = sqlite_utils.Database("simonwillisonblog.db")
    rag = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
    entries = list(db["blog_entry"].rows)
    entry_texts = [
        entry["title"] + '\n' + strip_html_tags(entry["body"])
        for entry in entries
    ]
    print("len of entry_texts is", len(entry_texts))
    entry_ids = [str(entry["id"]) for entry in entries]
    entry_metadatas = [
        {"slug": entry["slug"], "created": entry["created"]} for entry in entries
    ]
    rag.index(
        collection=entry_texts,
        document_ids=entry_ids,
        document_metadatas=entry_metadatas,
        index_name="blog",
        max_document_length=180,
        split_documents=True
    )


In [ ]:
go()

len of entry_texts is 3049
________________________________________________________________________________
WARNING! You have a GPU available, but only `faiss-cpu` is currently installed.
 This means that indexing will be slow. To make use of your GPU.
Please install `faiss-gpu` by running:
pip uninstall --y faiss-cpu & pip install faiss-gpu
 ________________________________________________________________________________
Will continue with CPU indexing in 5 seconds...


[Jan 29, 12:32:43] #> Note: Output directory .ragatouille/colbert/indexes/blog already exists


[Jan 29, 12:32:43] #> Will delete 1 files already at .ragatouille/colbert/indexes/blog in 20 seconds...
[Jan 29, 12:33:04] [0] 		 #> Encoding 9785 passages..
[Jan 29, 12:33:38] [0] 		 avg_doclen_est = 121.8592758178711 	 len(local_sample) = 9,785
[Jan 29, 12:33:38] [0] 		 Creating 16,384 partitions.
[Jan 29, 12:33:38] [0] 		 *Estimated* 1,192,393 embeddings.
[Jan 29, 12:33:38] [0] 		 #> Saving the indexing plan to .ragatouil

KeyboardInterrupt: 

In [1]:
from ragatouille import RAGPretrainedModel
rag = RAGPretrainedModel.from_index(".ragatouille/colbert/indexes/blog/")
docs = rag.search("what is shot scraper?")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading searcher for index blog for the first time... This may take a few seconds
[Jan 29, 12:53:34] #> Loading codec...
[Jan 29, 12:53:34] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jan 29, 12:53:34] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jan 29, 12:53:34] #> Loading IVF...
[Jan 29, 12:53:34] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1380.61it/s]

[Jan 29, 12:53:34] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00,  8.76it/s]

Searcher loaded!



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . what is shot scraper?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2003,  2915, 26988,  2099,  1029,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



In [2]:
docs

[{'content': "Running this:\r\n./blog-answer.sh 'What is shot-scraper?'\r\nOutputs this:\r\n\r\nShot-scraper is a Python utility that wraps Playwright, providing both a command line interface and a YAML-driven configuration flow for automating the process of taking screenshots of web pages and scraping data from them using JavaScript. It can be used to take one-off screenshots or take multiple screenshots in a repeatable way by defining them in a YAML file. Additionally, it can be used to execute JavaScript on a page and return the resulting value.\r\n\r\nThat's a really good description of my shot-scraper tool. I checked and none of that output is an exact match to content I had previously published on my blog.\r\nQ&amp;A\r\nMy talk ended with a Q&amp;A session. Here are the summarized questions and answers.\r\n\r\n\r\nHow does LangChain fit into this?",
  'score': 27.515625,
  'rank': 1,
  'document_id': '8296',
  'document_metadata': {'slug': 'embeddings',
   'created': '2023-10-23T

In [3]:
!pip install llm -q

In [4]:
!llm keys set openai


Enter key: 


In [5]:
open("/tmp/out.txt", "w").write(
    ' '.join([d['content']
    for d in rag.search("what is shot scraper?")]
))

7329

In [6]:
!cat /tmp/out.txt | llm -m gpt-4-turbo --system 'what is shot-scraper?'


Shot-scraper is a versatile and powerful Python tool designed for both web page screenshot capture and data scraping. It leverages Playwright to interact with web pages, allowing users to automate screenshot captures and execute JavaScript for data extraction. This tool provides a friendly command line interface and supports a YAML-driven configuration for defining multi-step screenshot and data scraping workflows. Developers and content creators can use shot-scraper to ensure their documentation contains up-to-date screenshots or to scrape web data for various applications, making it an invaluable asset in web development and content management projects.


In [23]:
colbert = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
docs = colbert.rerank(query='What is Datasette Lite?', documents=[
    d['content'] for d in docs
], k=5)

100%|██████████| 1/1 [00:00<00:00, 28.79it/s]


In [24]:
docs

[{'content': "shot-scraper --log-requests\r\nshot-scraper is my tool for automating screenshots, built on top of Playwright.\r\nIts latest feature was inspired by Datasette Lite.\r\nI have an ongoing ambition to get Datasette Lite to work entirely offline, using Service Workers.\r\nThe first step is to get it to work without loading external resources - it currently hits PyPI and a separate CDN multiple times to download wheels every time you load the application.\r\nTo do that, I need a reliable list of all of the assets that it's fetching.\r\nWouldn't it be handy If I could run a command and get a list of those resources?",
  'score': 21.125,
  'rank': 0,
  'result_index': 0},
 {'content': "Automating screenshots for the Datasette documentation using shot-scraper\nI released shot-scraper back in March as a tool for keeping screenshots in documentation up-to-date.\r\nIt's very easy for feature screenshots in documentation for a web application to drift out-of-date with the latest desi

In [25]:
open("/tmp/out1.txt", "w").write(
    ' '.join([d['content']
    for d in docs]
))

3394

In [26]:
!cat /tmp/out1.txt | llm -m gpt-4-turbo --system 'What is Datasette Lite?'


Datasette Lite is a version of Datasette, an open-source tool for exploring and publishing data, that runs entirely in the browser using WebAssembly. Datasette typically requires a server to run, but Datasette Lite aims to make it possible to work with datasets directly in the browser without needing any server-side components. This approach potentially makes Datasette accessible in environments where deploying a server is not possible or practical and could facilitate offline use of the tool via Service Workers. The ambition to make Datasette Lite work entirely offline highlights an effort to increase the tool's versatility and accessibility, making it easier for users to interact with and analyze data from anywhere, even without an internet connection.

The reference to Datasette Lite working offline and the desire to avoid loading external resources like packages from PyPI or assets from a CDN is part of an effort to make the application more self-contained and efficient. By reducin

In [12]:
import llm


In [16]:
model = llm.get_model("gpt-4-turbo")
model.key = ''


In [27]:
prompt = ' '.join([d['content']
    for d in docs])

In [35]:

response = model.prompt(
    prompt,
    system='What is Datasette Lite?'
)

In [36]:
for chunk in response:
    print(chunk, end="")

Datasette Lite is an experimental version of Datasette, a tool aimed at making it easier for journalists, researchers, and software developers to explore, analyze, and publish large datasets. While the original Datasette requires a server environment to run, Datasette Lite seeks to bring the core functionalities of Datasette to the web browser, leveraging technologies such as WebAssembly and SQLite.

The idea behind Datasette Lite is to run entirely in the client's browser, eliminating the need for server-side setup or execution. This makes Datasette more accessible for casual users or those who wish to quickly explore datasets without dealing with the complexities of server configuration or database management.

One of the goals for Datasette Lite, as mentioned, is to work entirely offline using Service Workers. This would allow users to access and interact with their data without an internet connection, once the initial load is complete. Achieving this involves ensuring that all nece